In [3]:
from lxml import etree
import numpy as np

In [4]:
# spherical along x, and revolute along y 

skyweave_mass = 2.1  # kg

z_offset = 1.2  # m

num_elements_x = 4
length_x = 0.8  # m
x_sep = length_x / (num_elements_x - 1) # the separation between each consecutive mass element in the x axis 

num_elements_y = 4
length_y = 0.8  # m
y_sep = length_y / (num_elements_y - 1) # the separation between each consecutive mass element in the y axis

uni_joint_x_stiffness = 10.5  # N/rad
uni_joint_y_stiffness = 10.5  # N/rad
uni_joint_x_damping = 0.1  # N/(rad/s)
uni_joint_y_damping = 0.1  # N/(rad/s)

num_elements = num_elements_x * num_elements_y
element_mass = skyweave_mass / num_elements

sdf = etree.Element("sdf", version="1.7")
robot = etree.SubElement(sdf, "model", name="spring_damper_mesh")
# pose = etree.SubElement(robot, "pose")
# pose.text = "0 0 0 0 0 0"
static = etree.SubElement(robot, "static")
static.text = "false"

# create a grid of point masses
pt_masses = []
for idx_y in range(num_elements_y):
    pt_masses.append([])
    for idx_x in range(num_elements_x):
        m = etree.SubElement(
            robot,
            "link",
            name=f"mass_x{idx_x}_y{idx_y}",
        )
        pose = etree.SubElement(m, "pose")
        pose.text = f"{(idx_x)*x_sep} {(idx_y)*y_sep} {z_offset} 0 0 0"
 
        inertial = etree.SubElement(m, "inertial")
        mass = etree.SubElement(inertial, "mass")
        mass.text = str(element_mass)
        inertia = etree.SubElement(inertial, "inertia")

        # define all inertia components and their values
        inertia_values = {
            "ixx": "1e-5", "ixy": "0", "ixz": "0",
            "iyy": "1e-5", "iyz": "0",
            "izz": "1e-5"
        }

        # create them in a loop
        for name, value in inertia_values.items():
            etree.SubElement(inertia, name).text = value

        collision = etree.SubElement(m, "collision", name=f"collision_x{idx_x}_y{idx_y}")
        geometry = etree.SubElement(collision, "geometry")
        sphere = etree.SubElement(geometry, "sphere")
        radius = etree.SubElement(sphere, "radius")
        radius.text = "0.03"

        visual = etree.SubElement(m, "visual", name=f"visual_x{idx_x}_y{idx_y}")
        geometry_v = etree.SubElement(visual, "geometry")
        sphere_v = etree.SubElement(geometry_v, "box")
        size_v = etree.SubElement(sphere_v, "size")
        size_v.text = "0.03 0.03 0.03"


        pt_masses[idx_y].append(m)

# link adjacent point masses with xy universal joints
for idx_y in range(num_elements_y):
    for idx_x in range(num_elements_x):
        # check if north connection is possible, if so create joint
        ## what are the conditions for a north connection to be possible?
        if idx_x < num_elements_x - 1:
            joint_north = etree.SubElement(
                robot,
                "joint",
                name=f"joint_x{idx_x}_y{idx_y}_to_x{idx_x+1}_y{idx_y}",
                type="ball",
            )
            parent = etree.SubElement(joint_north, "parent")
            parent.text = f"mass_x{idx_x}_y{idx_y}"
            child = etree.SubElement(joint_north, "child")
            child.text = f"mass_x{idx_x+1}_y{idx_y}"
            axis1 = etree.SubElement(joint_north, "axis")
            xyz1 = etree.SubElement(axis1, "xyz")
            xyz1.text = "0 1 0"
            dynamics1 = etree.SubElement(axis1, "dynamics")
            spring_stiffness = etree.SubElement(dynamics1, "spring_stiffness")
            spring_stiffness.text = str(uni_joint_x_stiffness)
            spring_reference = etree.SubElement(dynamics1, "spring_reference")
            spring_reference.text = "0"
            damping1 = etree.SubElement(dynamics1, "damping")
            damping1.text = str(uni_joint_x_damping)

 

            axis2 = etree.SubElement(joint_north, "axis2")
            xyz2 = etree.SubElement(axis2, "xyz")
            xyz2.text = "0 1 0"
            dynamics2 = etree.SubElement(axis2, "dynamics")
            spring_stiffness2 = etree.SubElement(dynamics2, "spring_stiffness")
            spring_stiffness2.text = str(uni_joint_y_stiffness)
            spring_reference = etree.SubElement(dynamics2, "spring_reference")
            spring_reference.text = "0"
            damping2 = etree.SubElement(dynamics2, "damping")
            damping2.text = str(uni_joint_y_damping)



            pose = etree.SubElement(joint_north, "pose", relative_to=f"mass_x{idx_x+1}_y{idx_y}")
            pose.text = f"{-0.5*x_sep} 0 0 0 0 0"

        # check if west connection is possible, if so, create joint
        if idx_y < num_elements_y - 1 and True:
            joint_west = etree.SubElement(
                robot,
                "joint",
                name=f"joint_x{idx_x}_y{idx_y}_to_x{idx_x}_y{idx_y+1}",
                type="revolute",
            )
            parent = etree.SubElement(joint_west, "parent")
            parent.text = f"mass_x{idx_x}_y{idx_y}"
            child = etree.SubElement(joint_west, "child")
            child.text = f"mass_x{idx_x}_y{idx_y+1}"
            axis1 = etree.SubElement(joint_west, "axis")
            xyz1 = etree.SubElement(axis1, "xyz")
            xyz1.text = "0 1 0"
            dynamics1 = etree.SubElement(axis1, "dynamics")
            spring_stiffness = etree.SubElement(dynamics1, "spring_stiffness")
            spring_stiffness.text = str(uni_joint_x_stiffness)
            spring_reference = etree.SubElement(dynamics1, "spring_reference")
            spring_reference.text = "0"
            damping1 = etree.SubElement(dynamics1, "damping")
            damping1.text = str(uni_joint_x_damping)



            axis2 = etree.SubElement(joint_west, "axis2")
            xyz2 = etree.SubElement(axis2, "xyz")
            xyz2.text = "0 1 0"
            dynamics2 = etree.SubElement(axis2, "dynamics")
            spring_stiffness2 = etree.SubElement(dynamics2, "spring_stiffness")
            spring_stiffness2.text = str(uni_joint_y_stiffness)
            spring_reference = etree.SubElement(dynamics2, "spring_reference")
            spring_reference.text = "0"
            damping2 = etree.SubElement(dynamics2, "damping")
            damping2.text = str(uni_joint_y_damping)



            pose = etree.SubElement(joint_west, "pose", relative_to=f"mass_x{idx_x}_y{idx_y+1}")
            pose.text = f"{-0.5*y_sep} 0 {z_offset} 0 0 {np.pi/2}"
    
# link diagonal point masses with shear springs attached to ball joints to prevent shear

#create a fixed joint between the first mass and the world
fixed_joint = etree.SubElement(
    robot,
    "joint",
    name="fixed_joint_mass_x0_y0_to_world",
    type="fixed",
)   
parent = etree.SubElement(fixed_joint, "parent")
parent.text = "world"
child = etree.SubElement(fixed_joint, "child")
child.text = "mass_x0_y0"
pose = etree.SubElement(fixed_joint, "pose", relative_to="mass_x0_y0")
pose.text = f"0 0 {z_offset} 0 0 0"

with open("spring_damper_mesh.sdf", "wb") as f:
    f.write(etree.tostring(sdf, pretty_print=True))

In [ ]:
# spherical along x, and y, with revolute springs along all diagonals

skyweave_mass = 2.1  # kg

z_offset = 1.2  # m

num_elements_x = 10
length_x = 0.8 # m
x_sep = length_x / (num_elements_x - 1) # the separation between each consecutive mass element in the x axis 

num_elements_y = 10
length_y = 0.8  # m
y_sep = length_y / (num_elements_y - 1) # the separation between each consecutive mass element in the y axis

uni_joint_x_stiffness = 0.02  # N/rad
uni_joint_y_stiffness = 0.02  # N/rad
uni_joint_x_damping = 0.001  # N/(rad/s)
uni_joint_y_damping = 0.001  # N/(rad/s)

num_elements = num_elements_x * num_elements_y
element_mass = skyweave_mass / num_elements

sdf = etree.Element("sdf", version="1.7")
robot = etree.SubElement(sdf, "model", name="spring_damper_mesh")
# pose = etree.SubElement(robot, "pose")
# pose.text = "0 0 0 0 0 0"
static = etree.SubElement(robot, "static")
static.text = "false"

# create a grid of point masses
pt_masses = []
for idx_y in range(num_elements_y):
    pt_masses.append([])
    for idx_x in range(num_elements_x):
        m = etree.SubElement(
            robot,
            "link",
            name=f"mass_x{idx_x}_y{idx_y}",
        )
        pose = etree.SubElement(m, "pose")
        pose.text = f"{(idx_x)*x_sep} {(idx_y)*y_sep} {z_offset} 0 0 0"
 
        inertial = etree.SubElement(m, "inertial")
        mass = etree.SubElement(inertial, "mass")
        mass.text = str(element_mass)
        inertia = etree.SubElement(inertial, "inertia")

        # define all inertia components and their values
        inertia_values = {
            "ixx": "1e-5", "ixy": "0", "ixz": "0",
            "iyy": "1e-5", "iyz": "0",
            "izz": "1e-5"
        }

        # create them in a loop
        for name, value in inertia_values.items():
            etree.SubElement(inertia, name).text = value

        collision = etree.SubElement(m, "collision", name=f"collision_x{idx_x}_y{idx_y}")
        geometry = etree.SubElement(collision, "geometry")
        sphere = etree.SubElement(geometry, "sphere")
        radius = etree.SubElement(sphere, "radius")
        radius.text = "0.03"

        visual = etree.SubElement(m, "visual", name=f"visual_x{idx_x}_y{idx_y}")
        geometry_v = etree.SubElement(visual, "geometry")
        sphere_v = etree.SubElement(geometry_v, "box")
        size_v = etree.SubElement(sphere_v, "size")
        size_v.text = "0.03 0.03 0.03"


        pt_masses[idx_y].append(m)

# link adjacent point masses with xy universal joints
for idx_y in range(num_elements_y):
    for idx_x in range(num_elements_x):
        # check if north connection is possible, if so create joint
        ## what are the conditions for a north connection to be possible?
        if idx_x < num_elements_x:
            joint_north = etree.SubElement(
                robot,
                "joint",
                name=f"N_x{idx_x}_y{idx_y}_to_x{idx_x+1}_y{idx_y}",
                type="ball",
            )

            parent = etree.SubElement(joint_north, "parent")
            parent.text = f"mass_x{idx_x}_y{idx_y}"
            child = etree.SubElement(joint_north, "child")
            child.text = f"mass_x{idx_x+1}_y{idx_y}"

            pose = etree.SubElement(joint_north, "pose", relative_to=f"mass_x{idx_x+1}_y{idx_y}")
            pose.text = f"{-0.5*x_sep} 0 0 0 0 0"

        # check if west connection is possible, if so, create joint
        if idx_y < num_elements_y and True:
            joint_west = etree.SubElement(
                robot,
                "joint",
                name=f"W_x{idx_x}_y{idx_y}_to_x{idx_x}_y{idx_y+1}",
                type="ball",
            )

            parent = etree.SubElement(joint_west, "parent")
            parent.text = f"mass_x{idx_x}_y{idx_y}"
            child = etree.SubElement(joint_west, "child")
            child.text = f"mass_x{idx_x}_y{idx_y+1}"

            pose = etree.SubElement(joint_west, "pose", relative_to=f"mass_x{idx_x}_y{idx_y+1}")
            pose.text = f"{-0.5*y_sep} 0 0 0 0 {np.pi/2}"

        # check if south west diagonal connection is possible, if so, create joint
        # check if idx_x is even, and idx y is odd, or if idx_x is odd and idx_y is even
        if (idx_x + idx_y) % 2 == 1 or True:
            # check if south west joint is possible
            if idx_x > 0 and idx_y < num_elements_y - 1:
                joint_sw = etree.SubElement(
                    robot,
                    "joint",
                    name=f"SW_x{idx_x}_y{idx_y}_to_x{idx_x-1}_y{idx_y+1}",
                    type="revolute",
                )
                parent = etree.SubElement(joint_sw, "parent")
                parent.text = f"mass_x{idx_x}_y{idx_y}"
                child = etree.SubElement(joint_sw, "child")
                child.text = f"mass_x{idx_x-1}_y{idx_y+1}"
                axis1 = etree.SubElement(joint_sw, "axis")
                xyz1 = etree.SubElement(axis1, "xyz")
                xyz1.text = "0 1 0"
                dynamics1 = etree.SubElement(axis1, "dynamics")
                spring_stiffness = etree.SubElement(dynamics1, "spring_stiffness")
                spring_stiffness.text = str(uni_joint_x_stiffness)
                spring_reference = etree.SubElement(dynamics1, "spring_reference")
                spring_reference.text = "0"
                damping1 = etree.SubElement(dynamics1, "damping")
                damping1.text = str(uni_joint_x_damping)

                pose = etree.SubElement(joint_sw, "pose", relative_to=f"mass_x{idx_x-1}_y{idx_y+1}")
                pose.text = f"{-0.5*((x_sep**2 + y_sep**2)**0.5)} 0 0 0 0 {0.75*np.pi}"

            #check if north west diagonal connection is possible, if so, create joint
            if idx_x < num_elements_x - 1 and idx_y < num_elements - 1:
                joint_nw = etree.SubElement(
                    robot,
                    "joint",
                    name=f"NW_x{idx_x}_y{idx_y}_to_x{idx_x+1}_y{idx_y+1}",
                    type="revolute",
                )
                parent = etree.SubElement(joint_nw, "parent")
                parent.text = f"mass_x{idx_x}_y{idx_y}"
                child = etree.SubElement(joint_nw, "child")
                child.text = f"mass_x{idx_x+1}_y{idx_y+1}"
                axis1 = etree.SubElement(joint_nw, "axis")
                xyz1 = etree.SubElement(axis1, "xyz")
                xyz1.text = "0 1 0"
                dynamics1 = etree.SubElement(axis1, "dynamics")
                spring_stiffness = etree.SubElement(dynamics1, "spring_stiffness")
                spring_stiffness.text = str(uni_joint_x_stiffness)
                spring_reference = etree.SubElement(dynamics1, "spring_reference")
                spring_reference.text = "0"
                damping1 = etree.SubElement(dynamics1, "damping")
                damping1.text = str(uni_joint_x_damping)
                pose = etree.SubElement(joint_nw, "pose", relative_to=f"mass_x{idx_x+1}_y{idx_y+1}")
                pose.text = f"{-0.5*((x_sep**2 + y_sep**2)**0.5)} 0 0 0 0 {np.pi/4}"
# link diagonal point masses with shear springs attached to ball joints to prevent shear

# create a fixed joint between masses and world where ind_y = 0
for ind_y in range(1):
    fixed_joint = etree.SubElement(
        robot,
        "joint",
        name=f"fixed_joint_mass_x0_y{ind_y}_to_world",
        type="fixed",
    )   
    parent = etree.SubElement(fixed_joint, "parent")
    parent.text = "world"
    child = etree.SubElement(fixed_joint, "child")
    child.text = f"mass_x0_y{ind_y}"
    # pose = etree.SubElement(fixed_joint, "pose", relative_to=f"mass_x0_y{ind_y}")
    # pose.text = f"0 0 {z_offset} 0 0 0"

with open("spring_damper_mesh.sdf", "wb") as f:
    f.write(etree.tostring(sdf, pretty_print=True))